# Project Description:
Arabic Review Sentiment Classification using LSTM and Gradio
Objective:
Build a sentiment analysis system for Arabic review texts using an Recurrent Neural Network (LSTM) to classify each review into one of three categories:
 - Negative (-1)
 - Neutral (0)
 - Positive (1)
The focus here is on leveraging sequential modeling capabilities of LSTM to handle text data more effectively.

The trained model should be deployed through a simple and user-friendly Gradio web application

# Install required libraries and Imports


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fahdseddik/arabic-company-reviews")

Using Colab cache for faster access to the 'arabic-company-reviews' dataset.


In [ ]:
!mkdir Data
!cp {path} -r Data

In [ ]:
!pip install pandas numpy gensim nltk tensorflow scikit-learn emoji gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 18.6 MB/s eta 0:00:00


In [ ]:
!pip install gensim

In [ ]:
!wget -c https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip
!unzip full_grams_cbow_100_twitter.zip
import gensim
import gensim.downloader as api
from gensim.models import Word2Vec

model_path = "full_grams_cbow_100_twitter.mdl"
w2v_model = Word2Vec.load(model_path)

--2025-12-10 07:55:16--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1131904811 (1.1G) [application/zip]
Saving to: ‘full_grams_cbow_100_twitter.zip’

full_grams_cbow_100 100%[===================>]   1.05G  56.1MB/s    in 33s     

2025-12-10 07:55:49 (32.7 MB/s) - ‘full_grams_cbow_100_twitter.zip’ saved [1131904811/1131904811]

Archive:  full_grams_cbow_100_twitter.zip
  inflating: full_grams_cbow_100_twitter.mdl  
  inflating: full_grams_cbow_100_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_grams_cbow_100_twitter.mdl.wv.vectors.npy  


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

# For tokenization and stemming
import nltk
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords

# For Word2Vec
from gensim.models import Word2Vec

# For Keras model
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# For evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


import re
import emoji
import gradio as gr
import pickle

import pandas as pd


# Read Data

In [ ]:
df = pd.read_csv("/content/Data/arabic-company-reviews/CompanyReviews.csv",index_col=0)
df.head()

,review_description,rating,company
0,رائع,1,talbat
1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,1,talbat
2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,-1,talbat
3,لماذا لا يمكننا طلب من ماكدونالدز؟,-1,talbat
4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,-1,talbat


#EDA

In [ ]:
print("Total samples:", len(df))

Total samples: 40046


In [ ]:
print("Rating distribution:\n", df['rating'].value_counts())

Rating distribution:
 rating
 1    23921
-1    14200
 0     1925
Name: count, dtype: int64


In [ ]:
df['text_len'] = df['review_description'].astype(str).apply(lambda x: len(x.split()))

In [ ]:
df['text_len']

,text_len
0,1
1,9
2,15
3,6
4,15
...,...
128,21
129,11
130,15
131,6


In [ ]:
print("Text length stats (words):\n", df['text_len'].describe())

Text length stats (words):
 count    40046.000000
mean         9.370774
std         12.527072
min          1.000000
25%          3.000000
50%          5.000000
75%         11.000000
max        443.000000
Name: text_len, dtype: float64


In [ ]:
# show top 10 longest reviews
print("\nTop long reviews:\n", df.sort_values('text_len', ascending=False).head(5)[['review_description','rating','text_len']])


Top long reviews:
                                       review_description  rating  text_len
38     الي/ السيد الاستاذ رئيس مجلس إدارة شركة طلعت م...      -1       443
8      ارجوا من سيادتكم التفضل بقراءة هذه السطورالموج...       1       374
5      امبارح بعد نص الليل كنت راجعة من الساحل المفرو...      -1       368
22286  خارا خرا خرا خرا خرا خرا خرا خرا خرا خرا خرا خ...      -1       258
4      عندهم امكانيات تخليهم احسن مطور عقاري في مصر و...      -1       235


In [ ]:
 # emoji counts sample (simple heuristic)
import re
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        "]+", flags=re.UNICODE)
df['emoji_count'] = df['review_description'].astype(str).apply(lambda s: len(emoji_pattern.findall(s)))
print("\nEmoji stats:\n", df['emoji_count'].describe())


Emoji stats:
 count    40046.000000
mean         0.114918
std          0.478353
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         44.000000
Name: emoji_count, dtype: float64


# Preprocessing

In [ ]:
NEGATIONS = {'لا','لم','لن','ما','مش','ليس','بدون','غير'}

arabic_norm_map = {
    'أ': 'ا', 'إ': 'ا', 'آ': 'ا',
    'ى': 'ي', 'ئ': 'ي', 'ؤ': 'و',
    'ة': 'ه', 'ﹷ': '', 'ـ': ''
}

In [ ]:
def normalize_arabic(text):
    # lower
    text = str(text).strip()
    # map characters
    for src, target in arabic_norm_map.items():
        text = text.replace(src, target)
    # remove tashkeel (diacritics)
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    # collapse repeated letters (optional)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    return text

In [ ]:
# regex to keep Arabic letters, Arabic-Indic digits? we will remove digits,
# keep emojis: pattern for emojis (we keep them)
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002700-\U000027BF"
        "]+", flags=re.UNICODE)

In [ ]:
# Tokenizer: keep Arabic letters and emojis and common punctuation inside words
arabic_token_pattern = re.compile(r'[\u0600-\u06FF]+|' + emoji_pattern.pattern)

In [ ]:
# compile regex for Latin + digits + punctuation to remove (we'll remove them)
non_arabic_regex = re.compile(r'[A-Za-z0-9\[\]\(\)\{\}\"«»<>@#$%^&*_+=~`;/\\|:<>.,؟؟،·؟…–\-]+')

In [ ]:
# load Arabic stopwords from NLTK then remove negations from that list
arabic_stopwords = set(stopwords.words("arabic"))

In [ ]:
# ensure negations are kept
arabic_stopwords = arabic_stopwords - NEGATIONS
arabic_stopwords

{'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آها',
 'آهاً',
 'آهٍ',
 'آهِ',
 'آي',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أف',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أقل',
 'أكتوبر',
 'أكثر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنى',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوه',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أينما',
 'أيها',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',


Stemmer

In [ ]:
stemmer = ISRIStemmer()

In [ ]:

def preprocess_text(text):
    """Normalize, keep emojis, remove latin/digits/punct, tokenize, remove stopwords (except negations), stem."""
    text = normalize_arabic(text)
    # extract emojis and arabic tokens
    tokens = arabic_token_pattern.findall(text)
    cleaned_tokens = []
    for tok in tokens:
        # if token is emoji (matches emoji pattern) keep as-is
        if emoji_pattern.match(tok):
            cleaned_tokens.append(tok)
            continue
        # remove non-arabic heavy noise inside tok
        tok = non_arabic_regex.sub(' ', tok)  # remove latin/digits/punct inside token
        tok = tok.strip()
        if tok == "":
            continue
        # split further by spaces if any
        for sub in tok.split():
            # drop stopwords except negations
            if sub in arabic_stopwords:
                continue
            # stem
            stemmed = stemmer.stem(sub)
            cleaned_tokens.append(stemmed)
    return cleaned_tokens


In [ ]:
# Quick test
print("preprocess sample:")
for s in df['review_description'].astype(str).head(5):
    print(s, "->", preprocess_text(s))

preprocess sample:
رائع -> ['ريع']
برنامج رائع جدا يساعد على تلبيه الاحتياجات بشكل اسرع -> ['رنمج', 'ريع', 'جدا', 'سعد', 'علي', 'لبه', 'حيج', 'شكل', 'سرع']
التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال بالشبكة..مع انه النت عندي تمام شو الحل?? -> ['طبق', 'لا', 'غتح', 'داي', 'يعط', 'لا', 'وجد', 'تصل', 'شبك', 'انه', 'الن', 'عند', 'تمم', 'شو', 'لحل']
لماذا لا يمكننا طلب من ماكدونالدز؟ -> ['لمذ', 'لا', 'يمك', 'طلب', 'ماكدونالدز']
البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكون فاتحه بقاله كده اكتر من شهر -> ['رنمج', 'ظهر', 'طعم', 'غلق', 'انه', 'بتك', 'فتح', 'بقل', 'كده', 'كتر', 'شهر']


# Build corpus for Word2Vec

In [ ]:
sentences = df['review_description'].astype(str).apply(preprocess_text).tolist()
sentences

[['ريع'],
 ['رنمج', 'ريع', 'جدا', 'سعد', 'علي', 'لبه', 'حيج', 'شكل', 'سرع'],
 ['طبق',
  'لا',
  'غتح',
  'داي',
  'يعط',
  'لا',
  'وجد',
  'تصل',
  'شبك',
  'انه',
  'الن',
  'عند',
  'تمم',
  'شو',
  'لحل'],
 ['لمذ', 'لا', 'يمك', 'طلب', 'ماكدونالدز'],
 ['رنمج',
  'ظهر',
  'طعم',
  'غلق',
  'انه',
  'بتك',
  'فتح',
  'بقل',
  'كده',
  'كتر',
  'شهر'],
 ['اصبح', 'غلي', 'جدا'],
 ['جمل', 'جدا', 'ريع'],
 ['اسف', 'وحد', 'صدم', 'زيد', 'سعر', 'اسف', 'طعم', 'صبح', 'عمل', 'ردي'],
 ['رنمج', 'ورز', 'وصل', 'احل', 'رنمج', 'فشل'],
 ['كتر',
  'تحس',
  'خدم',
  'عملاء',
  'لطف',
  'خطء',
  'قنه',
  'صلحو',
  'سرع',
  'بعض',
  'اخر',
  'سيس',
  'سوق',
  'فضل',
  'كرم',
  'نسب',
  'الي',
  'تفق'],
 ['متز'],
 ['سعر',
  'اغل',
  'محل',
  'كثر',
  'حطل',
  'وصل',
  'مجا',
  'حكي',
  'فضي',
  'طبق',
  'لا',
  'نصح'],
 ['طيبق', 'عمل', 'غدد'],
 ['سبت', 'احد', 'ربعاء', 'خمس'],
 ['اشل',
  'جدا',
  'بحث',
  'ما',
  'بعط',
  'فيد',
  'يتم',
  'كتب',
  'اسم',
  'طعم',
  'ومع',
  'بحث',
  'لا',
  'ظهر',
  'طعم',
 

Train Word2Vec

In [ ]:
# We'll train a modest Word2Vec on the corpus (if corpus is small, consider using pretrained Arabic embeddings)
w2v_size = 100
w2v_window = 5
w2v_min_count = 1  # set 2 or 3 on large corpora
w2v_workers = 4

w2v_model = Word2Vec(sentences=sentences, vector_size=w2v_size, window=w2v_window,
                     min_count=w2v_min_count, workers=w2v_workers, epochs=20)
print("Word2Vec vocabulary size:", len(w2v_model.wv.index_to_key))


Word2Vec vocabulary size: 13727


Tokenizer & sequences

In [ ]:
# we need a tokenizer that uses the same preprocessing logic
# but Keras Tokenizer expects strings; we will rejoin preprocessed tokens as space-separated strings
processed_texts = [" ".join(tokens) for tokens in sentences]

In [ ]:
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(processed_texts)
sequences = tokenizer.texts_to_sequences(processed_texts)

In [ ]:
# keep tokenizer mapping to words
word_index = tokenizer.word_index
print("Found", len(word_index), "unique tokens.")

Found 13728 unique tokens.


Create embedding matrix from Word2Vec

In [ ]:
vocab_size = min(20000, len(word_index) + 1)
embedding_dim = w2v_size
embedding_matrix = np.random.normal(size=(vocab_size, embedding_dim)).astype(np.float32) * 0.01
embedding_matrix

array([[-0.01293877, -0.00341697, -0.01950217, ..., -0.00864885,
         0.00108487, -0.01951495],
       [-0.00746429,  0.00466676, -0.0121663 , ..., -0.01968495,
        -0.0092833 ,  0.00682049],
       [-0.00112718,  0.00147304,  0.00578994, ...,  0.00790058,
        -0.00283381, -0.01476317],
       ...,
       [-0.00426151,  0.00446326, -0.00629687, ..., -0.00171713,
        -0.00539839,  0.00314058],
       [ 0.00969206,  0.00203742, -0.00712267, ..., -0.01632585,
        -0.01515681,  0.02038792],
       [-0.0148678 ,  0.00351833,  0.00450668, ..., -0.00802484,
        -0.00147145,  0.0139285 ]], dtype=float32)

In [ ]:
# fill embedding_matrix for words present in w2v
for word, i in word_index.items():
    if i >= vocab_size:
        continue
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
    # else keep random small vector (or zeros)

# Padding sequences

In [ ]:
maxlen = 80
X = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')

In [ ]:
# labels: convert rating (-1,0,1) to 0,1,2 for sparse_categorical_crossentropy
label_map = {-1: 0, 0: 1, 1: 2}
y = df['rating'].map(label_map).astype(int).values

# Train/test split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

In [ ]:
X_train.shape

(34039, 80)

# Build LSTM model with pretrained embedding

In [ ]:
num_classes = 3

model = Sequential()
# Input -> Embedding with pretrained weights from Word2Vec
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=True))  # set trainable to True to fine-tune embeddings

# LSTM with dropout and recurrent dropout
model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.2))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
#Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_lstm_arabic.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

In [ ]:
# Train
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10,
                    batch_size=32,
                    callbacks=callbacks)

Epoch 1/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - accuracy: 0.5969 - loss: 0.8341
Epoch 1: val_loss improved from inf to 0.71589, saving model to best_lstm_arabic.h5


1064/1064 ━━━━━━━━━━━━━━━━━━━━ 278s 254ms/step - accuracy: 0.5969 - loss: 0.8340 - val_accuracy: 0.6462 - val_loss: 0.7159
Epoch 2/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.6495 - loss: 0.7212
Epoch 2: val_loss did not improve from 0.71589
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 316s 253ms/step - accuracy: 0.6495 - loss: 0.7212 - val_accuracy: 0.5988 - val_loss: 0.8243
Epoch 3/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.6170 - loss: 0.8086
Epoch 3: val_loss did not improve from 0.71589
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 267s 251ms/step - accuracy: 0.6170 - loss: 0.8086 - val_accuracy: 0.6020 - val_loss: 0.8191
Epoch 4/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.7293 - loss: 0.6535
Epoch 4: val_loss improved from 0.71589 to 0.46524, saving model to best_lstm_arabic.h5


1064/1064 ━━━━━━━━━━━━━━━━━━━━ 267s 251ms/step - accuracy: 0.7293 - loss: 0.6534 - val_accuracy: 0.8312 - val_loss: 0.4652
Epoch 5/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.8391 - loss: 0.4476
Epoch 5: val_loss improved from 0.46524 to 0.45141, saving model to best_lstm_arabic.h5


1064/1064 ━━━━━━━━━━━━━━━━━━━━ 268s 252ms/step - accuracy: 0.8391 - loss: 0.4475 - val_accuracy: 0.8379 - val_loss: 0.4514
Epoch 6/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - accuracy: 0.8623 - loss: 0.4088
Epoch 6: val_loss did not improve from 0.45141
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 269s 253ms/step - accuracy: 0.8623 - loss: 0.4088 - val_accuracy: 0.8385 - val_loss: 0.4565
Epoch 7/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - accuracy: 0.8707 - loss: 0.3893
Epoch 7: val_loss did not improve from 0.45141
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 270s 254ms/step - accuracy: 0.8707 - loss: 0.3893 - val_accuracy: 0.8364 - val_loss: 0.4696
Epoch 8/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.8792 - loss: 0.3602
Epoch 8: val_loss did not improve from 0.45141
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 269s 252ms/step - accuracy: 0.8792 - loss: 0.3602 - val_accuracy: 0.8380 - val_loss: 0.4721
Epoch 8: early stopping
Restoring model weights from the end of the best epoch: 5.


Evaluate

In [ ]:
val_preds = model.predict(X_val)
y_pred = val_preds.argmax(axis=1)
print("Classification report on validation set:")
print(classification_report(y_val, y_pred, target_names=['neg','neu','pos']))

188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step
Classification report on validation set:
              precision    recall  f1-score   support

         neg       0.81      0.81      0.81      2130
         neu       0.00      0.00      0.00       289
         pos       0.86      0.92      0.89      3588

    accuracy                           0.84      6007
   macro avg       0.55      0.58      0.57      6007
weighted avg       0.80      0.84      0.82      6007



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print("Confusion matrix:\n", confusion_matrix(y_val, y_pred))

Confusion matrix:
 [[1729    0  401]
 [ 130    0  159]
 [ 284    0 3304]]


In [ ]:
# ---------- 14. Save artifacts ----------
model.save("arabic_sentiment_lstm_w2v.h5")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
w2v_model.save("w2v_arabic.model")

print("Saved model, tokenizer and w2v model.")

Saved model, tokenizer and w2v model.


In [ ]:
# gradio_app.py
import re
import pickle
import numpy as np
import tensorflow as tf
import gradio as gr
from gensim.models import Word2Vec
from nltk.stem.isri import ISRIStemmer

# load artifacts
model = tf.keras.models.load_model("arabic_sentiment_lstm_w2v.h5")
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)
w2v = Word2Vec.load("w2v_arabic.model")
stemmer = ISRIStemmer()

# same preprocessing functions used in training (copy them)
# (define normalize_arabic, emoji_pattern, arabic_token_pattern, non_arabic_regex, preprocess_text)
# to keep the answer concise, reuse the same preprocessing block you had in training.

# label mapping back
label_map_inv = {0: -1, 1: 0, 2: 1}

maxlen = 200

def preprocess_for_inference(text):
    tokens = preprocess_text(text)  # use same function as training
    joined = " ".join(tokens)
    seq = tokenizer.texts_to_sequences([joined])
    padded = np.array(tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=maxlen, padding='post'))
    return padded

def predict_fn(text):
    x = preprocess_for_inference(text)
    probs = model.predict(x)[0]
    pred_index = int(np.argmax(probs))
    sentiment = label_map_inv[pred_index]
    # prepare nice output
    labels = { -1: "Negative (-1)", 0: "Neutral (0)", 1: "Positive (1)"}
    return labels[sentiment], float(probs[pred_index])

# build Gradio UI
demo = gr.Interface(
    fn=predict_fn,
    inputs=gr.Textbox(lines=4, placeholder="ادخلي الريفيو العربي هنا..."),
    outputs=[gr.Textbox(label="Predicted sentiment"), gr.Number(label="Confidence")],
    title="Arabic Review Sentiment (LSTM + Word2Vec)",
    examples=[["الفيلم كان حلو جداً 😊"], ["التطبيق مش شغال خالص 😡"], ["عادي، لا شيء مميز."]]
)

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>